In [2]:
import pandas as pd


In [16]:
df = pd.read_csv("p2-texts/hansard40000.csv")

df['party'] = df['party'].replace({'Labour (Co-op)': 'Labour'})
# print(df['party'].unique())

party_counts = df['party'].value_counts()
main_parties = party_counts.nlargest(4).index.tolist()
df = df[df['party'].isin(main_parties)]
# print(df['party'].value_counts())

# 4th party 'Speaker'. Remove it.
df = df[df['party'] != 'Speaker']
# print(df['party'].value_counts())

df = df[df["speech_class"] == "Speech"]
speech_lengths = df["speech"].str.len()
speeches = speech_lengths >= 1000
df = df[speeches]

rows, columns = df.shape
print(f"Number of rows: {rows}")
print(f"Number of columns: {columns}")




Number of rows: 7815
Number of columns: 8
